# Moter dataset Anomaly detection

### Check dataset proportions

### dataset split 

To detect anomalies, we plan to input data into the model at 1-second intervals with a sampling rate of 100Hz. Therefore, we will segment the data into 70 samples for each 1-second intervals.

In [1]:
import pandas as pd

label_0 = pd.read_csv("data/label_0_full.csv")
label_1 = pd.read_csv("data/label_1_full.csv")
label_2 = pd.read_csv("data/label_2_full.csv")
label_3 = pd.read_csv("data/label_3_full.csv")

In [2]:
len_0 = len(label_0)
len_1 = len(label_1)
len_2 = len(label_2)
len_3 = len(label_3)

label_X = [label_0, label_1, label_2, label_3]
rem = [len_0, len_1, len_2, len_3]

for i in range(4):
    r = rem[i] % 100
    label_X[i] = label_X[i].iloc[:rem[i] - r]

    print(len(label_X[i]))



1445800
1440100
1414800
1439900


In [3]:
import numpy as np

In [4]:
data_split_0 = np.array_split(label_X[0], len(label_X[0]) // 100)
data_split_1 = np.array_split(label_X[1], len(label_X[1]) // 100)
data_split_2 = np.array_split(label_X[2], len(label_X[2]) // 100)
data_split_3 = np.array_split(label_X[3], len(label_X[3]) // 100)


y_label_0 = np.zeros(len(data_split_0))
y_label_1 = np.ones(len(data_split_1))
y_label_2 = np.full(len(data_split_2), 2)
y_label_3 = np.full(len(data_split_3), 3)

print(len(data_split_1), len(data_split_1[0]))


/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


14401 100


### Split train, validataion, test set

train_ratio = 0.7
validation ratio = 0.15
test ratio = 0.15

In [5]:
# 아래 코드 사용할 것.

train_ratio = 0.7
temp_ratio = 0.3
val_ratio = 0.5
test_ratio = temp_ratio / 2

data_splits = [data_split_0, data_split_1, data_split_2, data_split_3]
y_data_splits = [y_label_0, y_label_1, y_label_2, y_label_3]

train_sizes = []
train_data = []
test_data = []

y_train_data = []
y_test_data = []


for i in range(4):
    data_len = len(data_splits[i])
    train_size = int(data_len * train_ratio)
    train_sizes.append(train_size)
    train_data.append(data_splits[i][:train_size])
    test_data.append(data_splits[i][train_size:])

    y_train_data.append(y_data_splits[i][:train_size])
    y_test_data.append(y_data_splits[i][train_size:])


# check
for i in range(4):
    print(len(train_data[i]) + len(test_data[i]))

x_train_concat = np.concatenate(train_data, axis = 0)
x_test_concat = np.concatenate(test_data, axis = 0)
y_train_concat = np.concatenate(y_train_data, axis=0)
y_test_concat = np.concatenate(y_test_data, axis=0)


val_size = int(x_test_concat.shape[0] * val_ratio)
test_size = x_test_concat.shape[0] - val_size

# validation & test set split
x_val = x_test_concat[:val_size]
x_test = x_test_concat[val_size:]

y_val = y_test_concat[:val_size]
y_test = y_test_concat[val_size:]

# check
print(f"x_val size: {x_val.shape[0]}")
print(f"x_test size: {x_test.shape[0]}")
print(f"y_val size: {y_val.shape[0]}")
print(f"y_test size: {y_test.shape[0]}")

# check 
print(x_train_concat.shape[0] + x_test.shape[0] + x_val.shape[0])
print(y_train_concat.shape[0] + y_test.shape[0] + y_val.shape[0])

3599
3598
3599
3599
x_val size: 2160
x_test size: 2160
y_val size: 2160
y_test size: 2160
14395
14395


In [5]:
train_ratio = 0.7
temp_ratio = 0.3  # for both val and test combined
val_ratio = 0.5  # 50% of temp_ratio for validation
test_ratio = temp_ratio / 2  # same as val_ratio

data_splits = [data_split_0, data_split_1, data_split_2, data_split_3]
y_data_splits = [y_label_0, y_label_1, y_label_2, y_label_3]

train_sizes = []
train_data = []
val_data = []
test_data = []

y_train_data = []
y_val_data = []
y_test_data = []

for i in range(4):
    data_len = len(data_splits[i])
    train_size = int(data_len * train_ratio)
    temp_size = data_len - train_size 
    val_size = int(temp_size * val_ratio)
    
    # Split the data
    train_sizes.append(train_size)
    train_data.append(data_splits[i][:train_size])
    val_data.append(data_splits[i][train_size:train_size + val_size])
    test_data.append(data_splits[i][train_size + val_size:])

    # Split the labels
    y_train_data.append(y_data_splits[i][:train_size])
    y_val_data.append(y_data_splits[i][train_size:train_size + val_size])
    y_test_data.append(y_data_splits[i][train_size + val_size:])

# check the lengths of each split
for i in range(4):
    print(f"Data split {i}: train + val + test =", len(train_data[i]), "+", len(val_data[i]), "+", len(test_data[i]))

# Concatenate all splits across the 4 data parts
x_train_concat = np.concatenate(train_data, axis=0)
x_val_concat = np.concatenate(val_data, axis=0)
x_test_concat = np.concatenate(test_data, axis=0)

y_train_concat = np.concatenate(y_train_data, axis=0)
y_val_concat = np.concatenate(y_val_data, axis=0)
y_test_concat = np.concatenate(y_test_data, axis=0)

# Output final shapes to confirm correct splitting
print("Train data shape:", x_train_concat.shape)
print("Validation data shape:", x_val_concat.shape)
print("Test data shape:", x_test_concat.shape)


Data split 0: train + val + test = 10120 + 2169 + 2169
Data split 1: train + val + test = 10080 + 2160 + 2161
Data split 2: train + val + test = 9903 + 2122 + 2123
Data split 3: train + val + test = 10079 + 2160 + 2160
Train data shape: (40182, 100, 3)
Validation data shape: (8611, 100, 3)
Test data shape: (8613, 100, 3)


In [6]:
# Count the occurrences of each label in y_val_concat
unique_labels, label_counts = np.unique(y_val_concat, return_counts=True)

# Calculate the percentage of each label
label_ratios = label_counts / len(y_val_concat)

# Print the results
for label, count, ratio in zip(unique_labels, label_counts, label_ratios):
    print(f"Label {label}: count = {count}, ratio = {ratio:.2%}")


Label 0.0: count = 2169, ratio = 25.19%
Label 1.0: count = 2160, ratio = 25.08%
Label 2.0: count = 2122, ratio = 24.64%
Label 3.0: count = 2160, ratio = 25.08%


In [7]:
x_train_concat.shape

(40182, 100, 3)

In [8]:
y_train_concat.shape

(40182,)

In [9]:
from keras.utils import to_categorical

# Convert y_train and y_valid to one-hot encoding
y_train_one_hot = to_categorical(y_train_concat, num_classes=4)  # Assuming classes are 0, 1, 2, 3
y_valid_one_hot = to_categorical(y_val_concat, num_classes=4)  # Assuming you have a similar y_valid
y_test_one_hot = to_categorical(y_test_concat, num_classes=4)  # Assuming you have a similar y_test

# Check the shape
print(y_train_one_hot.shape)  # Should be (10070, 4)
print(y_valid_one_hot.shape) 
print(y_test_one_hot.shape) 

(40182, 4)
(8611, 4)
(8613, 4)


In [10]:
# Sum along axis 0 to get the count of each class
class_counts = np.sum(y_train_one_hot, axis=0)

# Calculate the percentage of each class
class_ratios = class_counts / np.sum(class_counts)

# Print the results
for i, (count, ratio) in enumerate(zip(class_counts, class_ratios)):
    print(f"Class {i}: count = {int(count)}, ratio = {ratio:.2%}")


Class 0: count = 10120, ratio = 25.19%
Class 1: count = 10080, ratio = 25.09%
Class 2: count = 9903, ratio = 24.65%
Class 3: count = 10079, ratio = 25.08%


### Define the Model Architecture

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15294558232718816693
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11378904359985974365
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6933955910558672802
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15573965249812017374
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13699355476082648183
physical_device_desc: "device: XLA_GPU device"
]


In [8]:

# from tensorflow.python.client import device_lib
# print(tf.__version__)
# print(device_lib.list_local_devices())


In [11]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dropout, Dense, InputLayer
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [20]:
model = keras.models.Sequential([
    # Conv Block 1 
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu",
                        input_shape=[100, 3]),
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu"), 

    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),

    # Conv Block 2 
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),           
    
    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 3
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),   
    
    # Pooling layer        
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 4
    # keras.layers.Conv1D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu"),

    keras.layers.GlobalAveragePooling1D(),

    # Updated output layer for multi-class classification
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(4, activation="softmax")  # Change to 4 units and softmax activation
])

# Compile the model
model.compile(loss="categorical_crossentropy",  # Use categorical crossentropy
              optimizer="adam",
              metrics=["accuracy"])  # Accuracy metric is still appropriate

# Train the model
history = model.fit(
    x_train_concat, y_train_one_hot,  # Ensure y_train is one-hot encoded for multi-class
    epochs=20,  
    batch_size=32,
    validation_data=(x_val_concat, y_valid_one_hot),
    verbose=1,
    # class_weight=class_weights_dict,
    # callbacks=[early_stop]
)

# Get predictions
y_pred = model.predict(x_val_concat)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

# # Save the model
model.save("conv_net_v1.h5")


/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7477 - loss: 0.5234 - val_accuracy: 0.8720 - val_loss: 0.3097
Epoch 2/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9417 - loss: 0.1490 - val_accuracy: 0.8425 - val_loss: 0.5430
Epoch 3/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9564 - loss: 0.1157 - val_accuracy: 0.8262 - val_loss: 0.7032
Epoch 4/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9777 - loss: 0.0629 - val_accuracy: 0.8243 - val_loss: 1.5073
Epoch 5/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9818 - loss: 0.0509 - val_accuracy: 0.8314 - val_loss: 0.9988
Epoch 6/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9870 - loss: 0.0391 - val_accuracy: 0.9475 - val_loss: 0.1342
Epoch 7/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9898 - loss: 0.0305 - val_accuracy: 0.9087 - val_loss: 0.2767
Epoch 8/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9909 - loss: 0.0273 -

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 100, 32)        │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 32)        │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 50, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 50, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 50, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 25, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 25, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 12, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,822 (2.61 MB)

 Trainable params: 227,940 (890.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 455,882 (1.74 MB)

In [21]:
loss, accuracy = model.evaluate(x_test_concat, y_test_one_hot, verbose=1)
print(loss)
print(accuracy)


270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9200 - loss: 0.2053
0.08895210176706314
0.9662138819694519


In [22]:
y_pred = model.predict(x_test_concat, verbose=1)

print(y_pred[:10])
print(y_test_concat[:10])

270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[[9.0344763e-01 4.5467289e-07 8.1638232e-02 1.4913647e-02]
 [9.9052703e-01 8.9956170e-10 8.3286809e-03 1.1442734e-03]
 [9.6998024e-01 1.3512960e-10 2.9757194e-02 2.6258337e-04]
 [4.7275230e-01 1.5128370e-06 3.8859278e-07 5.2724582e-01]
 [9.6528345e-01 2.2283839e-09 3.3260286e-02 1.4562479e-03]
 [9.7205019e-01 1.8445256e-07 1.3541399e-02 1.4408243e-02]
 [9.9707592e-01 7.3112644e-10 1.6090515e-03 1.3150580e-03]
 [9.9900717e-01 4.0546053e-11 4.3790665e-04 5.5498938e-04]
 [9.9698806e-01 1.1271378e-11 2.8719525e-03 1.3995037e-04]
 [9.7297227e-01 2.3191459e-08 7.3355593e-05 2.6954386e-02]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [26]:
import numpy as np

# y_pred의 형태 확인 및 정수형으로 변환
if y_pred.ndim == 2:
    y_pred = np.argmax(y_pred, axis=1)
elif y_pred.ndim == 1:
    print("y_pred는 이미 클래스 인덱스입니다.")

# y_test 및 y_pred의 고유 클래스 추출
unique_classes = np.unique(np.concatenate((y_test, y_pred)))  # 두 배열의 고유 클래스 합치기

# 클래스 인덱스를 0부터 시작하도록 조정
class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}
y_test_mapped = np.array([class_to_index[cls] for cls in y_test])
y_pred_mapped = np.array([class_to_index[cls] for cls in y_pred])


print(len(y_test_mapped))
print(len(y_pred_mapped))
# 혼동 행렬 초기화
num_classes = len(unique_classes)
conf_matrix = np.zeros((num_classes, num_classes), dtype=int)

# 혼동 행렬 계산
for i in range(len(y_test)):
    conf_matrix[y_test_mapped[i], y_pred_mapped[i]] += 1

# 혼동 행렬 출력
print("혼동 행렬:\n", conf_matrix)


2160
2160
혼동 행렬:
 [[  0   0   0   0]
 [  0   0   0   0]
 [540 540   0   0]
 [  0   0 540 540]]


In [34]:
len(y_test_concat)

2160

In [37]:
## v1 손대지 말것
from sklearn.metrics import confusion_matrix, classification_report
y_pred_argmax = np.argmax(y_pred, axis=1)

print(classification_report(y_test_concat, y_pred_argmax))

# 혼동 행렬 계산
cm = confusion_matrix(y_test_concat, y_pred_argmax)

# 혼동 행렬 출력
print("혼동 행렬:")
print(cm)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       540
         1.0       1.00      1.00      1.00       540
         2.0       1.00      1.00      1.00       540
         3.0       1.00      1.00      1.00       540

    accuracy                           1.00      2160
   macro avg       1.00      1.00      1.00      2160
weighted avg       1.00      1.00      1.00      2160

혼동 행렬:
[[540   0   0   0]
 [  0 540   0   0]
 [  0   0 540   0]
 [  0   0   0 540]]


In [53]:

## v2
from sklearn.metrics import confusion_matrix, classification_report
y_pred_argmax = np.argmax(y_pred, axis=1)

print(classification_report(y_test_concat, y_pred_argmax))

# 혼동 행렬 계산
cm = confusion_matrix(y_test_concat, y_pred_argmax)

# 혼동 행렬 출력
print("혼동 행렬:")
print(cm)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       540
         1.0       1.00      1.00      1.00       540
         2.0       1.00      1.00      1.00       540
         3.0       1.00      1.00      1.00       540

    accuracy                           1.00      2160
   macro avg       1.00      1.00      1.00      2160
weighted avg       1.00      1.00      1.00      2160

혼동 행렬:
[[540   0   0   0]
 [  0 540   0   0]
 [  0   0 540   0]
 [  0   0   0 540]]


In [23]:
## v3 손대지 말것
from sklearn.metrics import confusion_matrix, classification_report
y_pred_argmax = np.argmax(y_pred, axis=1)

print(classification_report(y_test_concat, y_pred_argmax))

# 혼동 행렬 계산
cm = confusion_matrix(y_test_concat, y_pred_argmax)

# 혼동 행렬 출력
print("혼동 행렬:")
print(cm)

              precision    recall  f1-score   support

         0.0       0.99      0.87      0.93      2169
         1.0       1.00      1.00      1.00      2161
         2.0       0.93      1.00      0.96      2123
         3.0       0.95      0.99      0.97      2160

    accuracy                           0.97      8613
   macro avg       0.97      0.97      0.97      8613
weighted avg       0.97      0.97      0.97      8613

혼동 행렬:
[[1894    0  154  121]
 [   0 2161    0    0]
 [   0    0 2123    0]
 [  16    0    0 2144]]


In [36]:
import tensorflow as tf
import numpy as np

# .h5 모델 로드
h5_model = tf.keras.models.load_model('conv_net_v1.h5')

# .tflite 변환기 생성
converter = tf.lite.TFLiteConverter.from_keras_model(h5_model)

# 양자화 적용 (정수 양자화)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter._experimental_disable_per_channel_quantization_for_dense_layers = True

# INT8 양자화를 위해 필요한 대표적인 데이터셋 함수 정의
def representative_dataset():
    for i in range(100):  # 임의로 100개의 샘플을 사용
        # X_windows_balanced에서 일부 샘플을 선택
        sample = x_train_concat[i]  # 각 sample의 shape은 (10, 3)
        # shape을 (1, 10, 3)로 맞추기 위해 배치 차원 추가
        yield [np.expand_dims(sample, axis=0).astype(np.float32)]

# # INT8 양자화를 위해 필요한 대표적인 데이터셋 함수 정의
# def representative_dataset():
#     for _ in range(100):
#         # (1, 10, 24)은 배치 크기 1, 10x24 크기의 입력 데이터를 의미합니다.
#         yield [np.random.rand(1, 10, 24).astype(np.float32)]

# # 대표 데이터셋 지정
converter.representative_dataset = representative_dataset

# 완전한 INT8 양자화를 위한 설정
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # INT8 연산만 허용
converter.inference_input_type = tf.int8  # 입력 타입을 INT8로 설정
converter.inference_output_type = tf.int8  # 출력 타입을 INT8로 설정


# .tflite 모델로 변환
tflite_model = converter.convert()

# .tflite 모델 저장
with open('conv_net_int8_v1_1.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpb9jgw9um/assets


INFO:tensorflow:Assets written to: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpb9jgw9um/assets


Saved artifact at '/var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpb9jgw9um'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  11223329152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223332848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223390288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223384480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223399792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223398912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223454064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223456352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223460576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223462864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11223464800: T

/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1727790421.593994 7865154 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727790421.594011 7865154 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-01 22:47:01.594132: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpb9jgw9um
2024-10-01 22:47:01.594739: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-01 22:47:01.594744: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpb9jgw9um
2024-10-01 22:47:01.601098: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-01 22:47:01.635

In [23]:
import numpy as np
from tensorflow import keras

def unet_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Encoder (Contracting Path)
    c1 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(inputs)
    c1 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(c1)
    p1 = keras.layers.MaxPooling1D(pool_size=2)(c1)

    c2 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(p1)
    c2 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(c2)
    p2 = keras.layers.MaxPooling1D(pool_size=2)(c2)

    c3 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(p2)
    c3 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(c3)
    p3 = keras.layers.MaxPooling1D(pool_size=2)(c3)

    c4 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(p3)
    c4 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(c4)
    p4 = keras.layers.MaxPooling1D(pool_size=2)(c4)

    # Bottleneck
    bottleneck = keras.layers.Conv1D(512, kernel_size=3, padding="same", activation="relu")(p4)
    bottleneck = keras.layers.Conv1D(512, kernel_size=3, padding="same", activation="relu")(bottleneck)

    # Decoder (Expanding Path)
    u4 = keras.layers.Conv1DTranspose(256, kernel_size=2, strides=2, padding="same")(bottleneck)
    u4 = keras.layers.concatenate([u4, c4])
    c5 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(u4)
    c5 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(c5)

    u3 = keras.layers.Conv1DTranspose(128, kernel_size=2, strides=2, padding="same")(c5)
    # c3_adjusted = keras.layers.Conv1D(128, kernel_size=1, padding="same")(c3) 
    c3_adjusted = keras.layers.Cropping1D(cropping=(1, 0))(c3)  # 마지막 1개 타임스텝 제거

    print(c3_adjusted.shape, u3.shape)
    u3 = keras.layers.concatenate([u3, c3_adjusted])
    c6 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(u3)
    c6 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(c6)

    u2 = keras.layers.Conv1DTranspose(64, kernel_size=2, strides=2, padding="same")(c6)
    c2_adjusted = keras.layers.Cropping1D(cropping=(2, 0))(c2)  # 마지막 1개 타임스텝 제거
    u2 = keras.layers.concatenate([u2, c2_adjusted])
    c7 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(u2)
    c7 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(c7)

    u1 = keras.layers.Conv1DTranspose(32, kernel_size=2, strides=2, padding="same")(c7)
    c1_adjusted = keras.layers.Cropping1D(cropping=(4, 0))(c1)  # 마지막 1개 타임스텝 제거
    u1 = keras.layers.concatenate([u1, c1_adjusted])
    c8 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(u1)
    c8 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(c8)

    outputs = keras.layers.Conv1D(4, kernel_size=1, activation="softmax")(c8)  # Change to 4 for multi-class output
    print(outputs.shape)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model

# Instantiate the model
input_shape = (100, 3)  # Modify based on your actual input shape
model = unet_model(input_shape)

# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# Train the model
history = model.fit(
    x_train_concat, y_train_one_hot,  # Ensure y_train is one-hot encoded for multi-class
    epochs=20,
    batch_size=32,
    validation_data=(x_val_concat, y_valid_one_hot),
    verbose=1,
    # class_weight=class_weights_dict,
    # callbacks=[early_stop]
)

# Get predictions
y_pred = model.predict(x_val_concat)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

# Save the model
model.save("u_net_v1.h5")


(None, 24, 128) (None, 24, 128)
(None, 96, 4)
Epoch 1/20


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 4), output.shape=(None, 96, 4)

In [25]:
model = keras.models.Sequential([
    # Conv Block 1 
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu",
                        input_shape=[100, 3]),
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu"), 

    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),

    # Conv Block 2 
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),           
    
    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 3
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),   
    
    # Pooling layer        
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 4
    # keras.layers.Conv1D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu"),

    keras.layers.GlobalAveragePooling1D(),

    # Updated output layer for multi-class classification
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(4, activation="sigmoid")  # Change to 4 units and softmax activation
])

# Compile the model
model.compile(loss="binary_crossentropy",  # Use categorical crossentropy
              optimizer="adam",
              metrics=["accuracy"])  # Accuracy metric is still appropriate

# Train the model
history = model.fit(
    x_train_concat, y_train_one_hot,  # Ensure y_train is one-hot encoded for multi-class
    epochs=20,  
    batch_size=32,
    validation_data=(x_val_concat, y_valid_one_hot),
    verbose=1,
    # class_weight=class_weights_dict,
    # callbacks=[early_stop]
)

# Get predictions
y_pred = model.predict(x_val_concat)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

# # Save the model
model.save("conv_net_v2.h5")


/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6625 - loss: 0.2973 - val_accuracy: 0.9133 - val_loss: 0.1051
Epoch 2/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9276 - loss: 0.0885 - val_accuracy: 0.7845 - val_loss: 0.5271
Epoch 3/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9497 - loss: 0.0623 - val_accuracy: 0.6471 - val_loss: 2.0219
Epoch 4/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9685 - loss: 0.0434 - val_accuracy: 0.6367 - val_loss: 7.0846
Epoch 5/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9754 - loss: 0.0342 - val_accuracy: 0.6574 - val_loss: 4.1364
Epoch 6/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9842 - loss: 0.0220 - val_accuracy: 0.6193 - val_loss: 0.8876
Epoch 7/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9843 - loss: 0.0223 - val_accuracy: 0.8033 - val_loss: 0.5506
Epoch 8/20
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9880 - loss: 0.0170 - 

In [28]:
y_pred

array([[9.99945641e-01, 4.95489029e-17, 2.61610154e-07, 1.38890509e-05],
       [9.99852240e-01, 9.63207010e-16, 4.83867495e-07, 5.63673239e-05],
       [9.92969811e-01, 6.21100463e-11, 5.78368963e-06, 5.58351120e-03],
       ...,
       [1.16577063e-07, 1.35192211e-04, 7.40288973e-20, 9.99954045e-01],
       [1.05082984e-06, 1.21905752e-04, 2.65338427e-18, 9.99901235e-01],
       [6.39405195e-10, 1.05226005e-04, 1.20326106e-23, 9.99913812e-01]],
      dtype=float32)

In [32]:
loss, accuracy = model.evaluate(x_test_concat, y_test_one_hot, verbose=1)
print(loss)
print(accuracy)

270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4077 - loss: 2.3237
0.9807576537132263
0.7492163181304932


In [33]:
## v4 손대지 말것 sigmoid
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# 예측값 얻기
y_pred = model.predict(x_test_concat)

# 특정 임계값(예: 0.5)을 기준으로 레이블 변환
threshold = 0.5
y_pred_labels = (y_pred >= threshold).astype(int)

# 실제 레이블(y_true)도 이진 형식이어야 합니다.
# 예: y_true = [[0, 1, 0, 0], [1, 0, 1, 0], ...]

# 혼동 행렬 계산
# 실제 레이블과 예측된 레이블을 flatten하여 사용
cm = confusion_matrix(y_test_concat.flatten(), y_pred_labels.flatten())

# 혼동 행렬 시각화
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

270/270 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


ValueError: Found input variables with inconsistent numbers of samples: [8613, 34452]

In [34]:
# softmax without
# 기존 모델 로드
model = keras.models.load_model("conv_net_v1.h5")

# 모델 호출로 입력을 정의합니다.
example_input = np.random.rand(1, 100, 3)  # 예제 입력 정의
_ = model(example_input)  # 모델 호출하여 입력을 정의합니다.

# 소프트맥스 레이어 제외한 모델 생성
model_without_softmax = keras.models.Model(inputs=model.inputs, outputs=model.layers[-2].output)

# 확인
model_without_softmax.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model_without_softmax)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter._experimental_disable_per_channel_quantization_for_dense_layers = True



# INT8 양자화를 위해 필요한 대표적인 데이터셋 함수 정의
def representative_dataset():
    for i in range(100):  # 임의로 100개의 샘플을 사용
        # X_windows_balanced에서 일부 샘플을 선택
        sample = x_train_concat[i]  # 각 sample의 shape은 (10, 3)
        # shape을 (1, 10, 3)로 맞추기 위해 배치 차원 추가
        yield [np.expand_dims(sample, axis=0).astype(np.float32)]

converter.representative_dataset = representative_dataset

# 완전한 INT8 양자화를 위한 설정
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # INT8 연산만 허용
converter.inference_input_type = tf.int8  # 입력 타입을 INT8로 설정
converter.inference_output_type = tf.int8  # 출력 타입을 INT8로 설정


# .tflite 모델로 변환
tflite_model = converter.convert()


# 6. TFLite 모델 저장
with open('conv_net_model_without_softmax.tflite', 'wb') as f:
    f.write(tflite_model)

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 3)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 100, 32)        │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 100, 32)        │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 50, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 50, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 50, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 25, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_12 (Conv1D)              │ (None, 25, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,480 (439.38 KB)

 Trainable params: 112,480 (439.38 KB)

 Non-trainable params: 0 (0.00 B)

INFO:tensorflow:Assets written to: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpwxp4mwsg/assets


INFO:tensorflow:Assets written to: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpwxp4mwsg/assets


Saved artifact at '/var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpwxp4mwsg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 128), dtype=tf.float32, name=None)
Captures:
  11171954384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172433584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172441680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172444144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172467936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172470928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172473568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172475856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172478320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172679168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  11172681104:

/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1727790329.394891 7865154 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727790329.395834 7865154 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-01 22:45:29.397741: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpwxp4mwsg
2024-10-01 22:45:29.398583: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-01 22:45:29.398589: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpwxp4mwsg
2024-10-01 22:45:29.407616: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-01 22:45:29.493